In [1]:
import challenge1000
import DA_IO

In [2]:
X = challenge1000.Native().X
ALL = challenge1000.Native().ALL

/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/SCHEMA/star_schema.json
/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/SCHEMA/star_schema.json
/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/SCHEMA/star_schema.json
/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/SCHEMA/star_schema.json


In [3]:
X.keys()

Index(['nom_struc', 'categorie', 'cat_struc', 'cat_autre_struc', 'age_pers',
       'nbr_salarie', 'ca_2017', 'ca_2018', 'ca_2019', 'pays_struc1',
       'pays_struc2', 'date_struc', 'prix_struc', 'linkedin_struc',
       'email_pers'],
      dtype='object')

In [4]:
set(X['categorie'])

{'agriculture',
 'autres',
 'culture',
 'ecommerce',
 'education',
 'energie',
 'finance',
 'gouvernement',
 'media',
 'mobilite',
 'non renseigné',
 'sante',
 'technologie'}

In [5]:
critere_secteur = ['agriculture','energie','education', 'sante', 'finance', 'mobilite', 'technologie', 'non renseigné']

X['critere_secteur'] = X['categorie'].map(lambda x: True if x in critere_secteur else False)

In [6]:
not_africa = ['Bermuda', 'United States','Suisse', 'Singapore', 'Royaume-Uni', 'Pakistan', 'Lebanon', 'Hong Kong', 'France', 'Germany', 'Afghanistan']
X['critere_geographique_1'] = X['pays_struc1'].map(lambda x: True if x not in not_africa else False)
X['critere_geographique_2'] = X['pays_struc2'].map(lambda x: True if x not in not_africa else False)
X['critere_geographique'] = X['critere_geographique_1'] & X['critere_geographique_2']

In [7]:
X['critere_ca_2017'] = X['ca_2017'].map(lambda x: True if x>200000 else False)
X['critere_ca_2018'] = X['ca_2018'].map(lambda x: True if x>200000 else False)
X['critere_ca_2019'] = X['ca_2019'].map(lambda x: True if x>200000 else False)
X['critere_ca'] = X['critere_ca_2017'] & X['critere_ca_2018'] & X['critere_ca_2019']

In [8]:
import datetime
import dateutil.relativedelta
def minus18(date_string):
    d = datetime.datetime.strptime("2020-03-31", "%Y-%m-%d")
    d2 = d - dateutil.relativedelta.relativedelta(months=1)
    return d2

def diff_month(d2):
    d1 = datetime.datetime.strptime("2020-10-01", "%Y-%m-%d")
    d2 = datetime.datetime.strptime(d2, "%Y-%m-%d")
    return (d1.year - d2.year) * 12 + d1.month - d2.month
ALL['months'] = ALL['date_struc'].map(diff_month)
ALL['critere_months'] = ALL['months'].map(lambda x: True if x>= 18 else False)

In [9]:
ALL[ALL['critere_months']][['date_struc','months', 'critere_months']].sort_values('months')

,date_struc,months,critere_months
key_main,,,
https://sovise-cook-bake.business.site/_patrickatrogo_sophia,2019-04-26,18,True
http://www.workload.ng_izah_nelson,2019-04-17,18,True
http://www.caelklima.com_maingey_yvonne,2019-04-10,18,True
http://www.aquareignindustry.com_mbouleediango_marcidriss,2019-04-24,18,True
http://www.maeimmobilier.com_ekue_ayelemichellelorraine,2019-04-12,18,True
...,...,...,...
http://www.manutafci.com_pahidewinde_bdanielomer,1941-01-24,957,True
http://www.ministry%20of%20sport.com_akram_hania,1930-01-31,1089,True
http://www.ministry%20of%20youth%20and%20sports.com_reen_hatem,1930-01-08,1089,True


In [10]:
ranking_SU_47 = DA_IO.Gspreadsheet('ranking_SU_47').sheet_to_df().set_index('key_main')

Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com


In [11]:
[a for a in ALL.keys() if 'critere' in a]

['critere_months']

In [12]:
master = X[['critere_secteur', 'critere_geographique_1', 'critere_geographique_2', 'critere_geographique', 'critere_ca_2017', 'critere_ca_2018', 'critere_ca_2019', 'critere_ca']].join(ALL['critere_months'])#.join(ranking_SU_47).dropna()

In [13]:
master['all_critere'] = master['critere_secteur'] & master['critere_geographique'] & master['critere_ca'] & master['critere_months']
master['all_critere_ca2019'] = master['critere_secteur'] & master['critere_geographique'] & master['critere_ca_2019'] & master['critere_months']

In [14]:
all_critere = master[master['all_critere']]
all_critere

,critere_secteur,critere_geographique_1,critere_geographique_2,critere_geographique,critere_ca_2017,critere_ca_2018,critere_ca_2019,critere_ca,critere_months,all_critere,all_critere_ca2019
key_main,,,,,,,,,,,
http://www.ucma.com_mohamed_abdallah,True,True,True,True,True,True,True,True,True,True,True
http://www.jobartis.com_verdeja_luis,True,True,True,True,True,True,True,True,True,True,True
http://www.kilimo.org_wostry_janetmaro,True,True,True,True,True,True,True,True,True,True,True
http://www.citmindustries.com_kouassi_konanhubert,True,True,True,True,True,True,True,True,True,True,True
http://www.yofcing.org_anwan_ukeme,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
http://www.espacendugu.com_kane_ahmadou,True,True,True,True,True,True,True,True,True,True,True
http://www.itserv.tn_elabed_imed,True,True,True,True,True,True,True,True,True,True,True
https://ozow.com/_adamseckstein_mitchanlyle,True,True,True,True,True,True,True,True,True,True,True


In [15]:
notes_all_critere = master[master['all_critere']].join(ranking_SU_47).dropna()
notes_all_critere

,critere_secteur,critere_geographique_1,critere_geographique_2,critere_geographique,critere_ca_2017,critere_ca_2018,critere_ca_2019,critere_ca,critere_months,all_critere,...,Solution numérique - Moy,FG - Tech4good,PT - Tech4Good,AA - Tech4Good,Tech4Good - Moy,MOYENNE NOTE,RESILIENT,Dans vertical N°1 :e-santé; mobilité; agriculture et food tech; \nlogistique; edtech; travail de demain; e-commerce; service financier; energie; environement,Dans vertical N°2 (si pertinent),Commentaires
key_main,,,,,,,,,,,,,,,,,,,,,
http://www.brastorne.com_martin_stimela,True,True,True,True,True,True,True,True,True,True,...,"4,333333333",5,5,4,"4,666666667","4,5",2,agriculture et food tech,tous,
http://www.havenhillsynergy.com_odunaiya_olusegun,True,True,True,True,True,True,True,True,True,True,...,"0,6666666667",3,4,3,"3,333333333",2,0,energie,,
http://www.snapnsave.co.za_fisher_tina,True,True,True,True,True,True,True,True,True,True,...,"4,666666667",1,2,1,"1,333333333",3,0,e-commerce,,
https://ozow.com/_adamseckstein_mitchanlyle,True,True,True,True,True,True,True,True,True,True,...,"4,666666667",4,4,4,4,"4,333333333",1,service financier,,
https://rabboni.co.ug/_makumbi_henry,True,True,True,True,True,True,True,True,True,True,...,"3,666666667",3,3,4,"3,333333333","3,5",0,agriculture et food tech\n,,Pas pertinent
https://www.geodatadesign.co.za/_mentz_valerie,True,True,True,True,True,True,True,True,True,True,...,"4,666666667",3,2,2,"2,333333333","3,5",1,environement,,


In [16]:
all_critere_ca2019 = master[master['all_critere_ca2019']]
all_critere_ca2019

,critere_secteur,critere_geographique_1,critere_geographique_2,critere_geographique,critere_ca_2017,critere_ca_2018,critere_ca_2019,critere_ca,critere_months,all_critere,all_critere_ca2019
key_main,,,,,,,,,,,
http://www.ucma.com_mohamed_abdallah,True,True,True,True,True,True,True,True,True,True,True
http://www.jobartis.com_verdeja_luis,True,True,True,True,True,True,True,True,True,True,True
http://www.volcanocoffeeltd.com_gerald_katabazi,True,True,True,True,False,False,True,False,True,False,True
http://MHFM.BUSINESS.SITE_mwakyambiki_jacqueline,True,True,True,True,False,True,True,False,True,False,True
http://www.visit-cameroun.com_mamoudou_nene,True,True,True,True,False,True,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
http://www.mpetchad.com_youssoufrahma_amane,True,True,True,True,True,False,True,False,True,False,True
http://berag-ziane.com_ziane_hafidhou,True,True,True,True,False,False,True,False,True,False,True
http://www.cbdibaong.org_bigo_sedjrogwladysbertille,True,True,True,True,False,True,True,False,True,False,True


In [17]:
notes_all_critere_ca2019 = master[master['all_critere_ca2019']].join(ranking_SU_47).dropna()
notes_all_critere_ca2019

,critere_secteur,critere_geographique_1,critere_geographique_2,critere_geographique,critere_ca_2017,critere_ca_2018,critere_ca_2019,critere_ca,critere_months,all_critere,...,Solution numérique - Moy,FG - Tech4good,PT - Tech4Good,AA - Tech4Good,Tech4Good - Moy,MOYENNE NOTE,RESILIENT,Dans vertical N°1 :e-santé; mobilité; agriculture et food tech; \nlogistique; edtech; travail de demain; e-commerce; service financier; energie; environement,Dans vertical N°2 (si pertinent),Commentaires
key_main,,,,,,,,,,,,,,,,,,,,,
http://MHFM.BUSINESS.SITE_mwakyambiki_jacqueline,True,True,True,True,False,True,True,False,True,False,...,3,3,3,4,"3,333333333","3,166666667",0,agriculture et food tech,,Pas pertinent\n\nAA\n\nPourquoi pas pertinent ?
http://www.MandulisEnergy.com_nyeko_peterbenhurodokonyero,True,True,True,True,False,False,True,False,True,False,...,"0,3333333333",4,4,4,4,"2,166666667",2,énergie,,
http://www.brastorne.com_martin_stimela,True,True,True,True,True,True,True,True,True,True,...,"4,333333333",5,5,4,"4,666666667","4,5",2,agriculture et food tech,tous,
http://www.coldhubs.com_ikegwuonu_nnaemeka,True,True,True,True,False,False,True,False,True,False,...,0,4,5,4,"4,333333333","2,166666667",2,logistique,agriculture et food tech,
http://www.havenhillsynergy.com_odunaiya_olusegun,True,True,True,True,True,True,True,True,True,True,...,"0,6666666667",3,4,3,"3,333333333",2,0,energie,,
http://www.mellowvans.com_dupreez_neil,True,True,True,True,False,True,True,False,True,False,...,"0,6666666667",2,3,3,"2,666666667","1,666666667",0,mobilité,,
http://www.mymdoc.com_mobisson_nneka,True,True,True,True,False,False,True,False,True,False,...,5,5,5,5,5,5,2,e-santé,,
http://www.paysky.io_sadek_waleed,True,True,True,True,False,True,True,False,True,False,...,"4,666666667",3,3,3,3,"3,833333333",1,service financier,,
http://www.snapnsave.co.za_fisher_tina,True,True,True,True,True,True,True,True,True,True,...,"4,666666667",1,2,1,"1,333333333",3,0,e-commerce,,


In [18]:
headers2 = ['nom_struc','categorie','pays_struc1','pays_struc2', 'ca_2017', 'ca_2018', 'ca_2019', 'months','prez_struc', 'prez_produit_struc']
headers = ['investisseur_struc','nom_pers', 'prenom_pers','email_pers', 'mobile_pers', 'fixe_pers']
headers3 = ['FG - Solution numérique', 'PT - Solution numérique', 'AA - Solution numérique', 'Solution numérique - Moy']

In [19]:
export_all_critere = all_critere.join(ALL[headers2+headers])[headers2+headers]
export_all_critere

,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,ca_2019,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,email_pers,mobile_pers,fixe_pers
key_main,,,,,,,,,,,,,,,,
http://www.ucma.com_mohamed_abdallah,Union maraichair de cooperative agricole d'agadez,non renseigné,Niger,Niger,585000.0,700000.0,700000.0,191,La cooperative et pour but d'ameliorè le rende...,L'oignon pomme de terr son le precipal activit...,La cooperative elle meme,Mohamed,Abdallah,www.mohamedabdallahhouma@gmail.com,+22797730729,+227
http://www.jobartis.com_verdeja_luis,SAGINV Lda,education,Angola,Spain,400000.0,700000.0,850000.0,83,"Launched in 2013, Jobartis is the main online ...",Online Job Board: www.jobartis.com\r\nEducatio...,Business Angels\r\nThe Venture City,Verdeja,Luis,accounts@jobartis.com,+244 933 393 226,NaN
http://www.kilimo.org_wostry_janetmaro,Sustainable Agriculture Tanzania (SAT),agriculture,Tanzania,Tanzania,1000000.0,1200000.0,1500000.0,112,Sustainable Agriculture Tanzania (SAT) is a gr...,At SAT we have build our competence in the fie...,NaN,Wostry,Janet Maro,janet.maro@kilimo.org,+255754925560,NaN
http://www.citmindustries.com_kouassi_konanhubert,Compagnie Ivoirienne de Travaux Métallique et ...,education,Côte d'Ivoire,Côte d'Ivoire,260000.0,275000.0,465000.0,93,CITM (compagnie ivoirienne de Travaux Métalliq...,"Dans mon pays , les entreprises évoluant dans ...",Fond propre,KOUASSI,Konan Hubert,hubert.kouassi@citmindustries.com,22507065728,22521275135
http://www.yofcing.org_anwan_ukeme,Youth for Change and Social Life Development F...,agriculture,Nigeria,Nigeria,11664000.0,116640000.0,92448000.0,71,"YOUTH FOR CHANGE INITIATIVE,Youth for change &...","OCTOBER, 2018\r\nSTUDENT ORIENTATION SUMMIT\r\...",NIL,Anwan,Ukeme,ukemeanwan2@gmail.com,+2347031177409,+2348084943878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://www.espacendugu.com_kane_ahmadou,BASIF,non renseigné,Sénégal,Sénégal,223425.0,269090.0,548000.0,131,BASIF – SARL comme son nom l’indique est une...,L’entreprise BASIF souhaite étendre son expéri...,NaN,KANE,Ahmadou,kaneahmadou@hotmail.com,+221771608921,+221771608921
http://www.itserv.tn_elabed_imed,IT SERV,sante,Tunisie,Tunisie,850000.0,1000000.0,865000.0,150,IT SERV est un groupe de sociétés dans le numé...,2 offres phares :\r\n- Domaine Télécoms : OSS ...,Personnes physiques,ELABED,Imed,imed.elabed@itserv.tn,+21698338339,+21671230206
https://ozow.com/_adamseckstein_mitchanlyle,Ozow,finance,South Africa,South Africa,334524.0,1175854.0,2175878.0,74,Ozow is the future of payment as we know it. A...,An automated EFT solution that helps customers...,"Kalon Venture Capital, Vunani, Buffet Investme...",Adams-----Eckstein,Mitchan-----Lyle,mitch@ozow.com-----lyle@ozow.com,+27 82 400 4423,+27 11 054 4744


In [20]:
export_notes_all_critere = notes_all_critere.join(ALL[headers2+headers], lsuffix = '_')[headers3+ headers2+headers]
export_notes_all_critere

,FG - Solution numérique,PT - Solution numérique,AA - Solution numérique,Solution numérique - Moy,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,ca_2019,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,email_pers,mobile_pers,fixe_pers
key_main,,,,,,,,,,,,,,,,,,,,
http://www.brastorne.com_martin_stimela,4,4,5,"4,333333333",Brastorne enterprises,agriculture,Botswana,Botswana,350000.0,570000.0,599250.0,82,Brastorne Enterprises is a 100% youth owned co...,Traditionally known for mobile wallets and pay...,NaN,Martin,Stimela,martin@brastorne.com,+26772820009,+2673114683
http://www.havenhillsynergy.com_odunaiya_olusegun,0,1,1,"0,6666666667",HAVENHILL SYNERGY LIMITED,energie,Nigeria,Nigeria,282466.0,210570.0,404069.0,128,Havenhill Synergy is a clean-tech micro utilit...,"Specifically, we have built and operating a 2...",USADF: United States African Development Found...,Odunaiya,Olusegun,corporate@havenhillsynergy.com,+2348127736184,NaN
http://www.snapnsave.co.za_fisher_tina,5,5,4,"4,666666667",SnapnSave,finance,South Africa,South Africa,214000.0,393000.0,640000.0,61,SnapnSave is South Africa’s #1 cashback coupon...,"In just over 4 years, SnapnSave has revolution...",Kalon Venture Partners\r\nVunani Investsments ...,Fisher,Tina,tina@snapnsave.co.za,+27784219892,NaN
https://ozow.com/_adamseckstein_mitchanlyle,4,5,5,"4,666666667",Ozow,finance,South Africa,South Africa,334524.0,1175854.0,2175878.0,74,Ozow is the future of payment as we know it. A...,An automated EFT solution that helps customers...,"Kalon Venture Capital, Vunani, Buffet Investme...",Adams-----Eckstein,Mitchan-----Lyle,mitch@ozow.com-----lyle@ozow.com,+27 82 400 4423,+27 11 054 4744
https://rabboni.co.ug/_makumbi_henry,4,4,3,"3,666666667",Rabboni Group Ltd,agriculture,Uganda,Uganda,303630.0,508966.0,1082769.0,70,Rabboni Group Ltd is an agricultural developme...,Value addition for maize grain into human food...,KIVA (Crowd funding platform).\r\nFACTS (Finan...,Makumbi,Henry,henry@rabboni.co.ug,+256752436536,+256392962193
https://www.geodatadesign.co.za/_mentz_valerie,4,5,5,"4,666666667",GEO Data Design,technologie,South Africa,South Africa,1000000.0,1100000.0,1200000.0,245,GEO Data Design helps organisations across Afr...,Prometeo™ is a satellite-based vegetation fire...,Private Company,Mentz,Valerie,val@datadesign.co.za,+(27) 82 554 5511,+(27) 21 425 9833


In [21]:
export_all_critere_ca2019 = all_critere_ca2019.join(ALL[headers2+headers])[headers2+headers]
export_all_critere_ca2019

,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,ca_2019,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,email_pers,mobile_pers,fixe_pers
key_main,,,,,,,,,,,,,,,,
http://www.ucma.com_mohamed_abdallah,Union maraichair de cooperative agricole d'agadez,non renseigné,Niger,Niger,585000.0,700000.0,700000.0,191,La cooperative et pour but d'ameliorè le rende...,L'oignon pomme de terr son le precipal activit...,La cooperative elle meme,Mohamed,Abdallah,www.mohamedabdallahhouma@gmail.com,+22797730729,+227
http://www.jobartis.com_verdeja_luis,SAGINV Lda,education,Angola,Spain,400000.0,700000.0,850000.0,83,"Launched in 2013, Jobartis is the main online ...",Online Job Board: www.jobartis.com\r\nEducatio...,Business Angels\r\nThe Venture City,Verdeja,Luis,accounts@jobartis.com,+244 933 393 226,NaN
http://www.volcanocoffeeltd.com_gerald_katabazi,Volcano Coffee Limited,agriculture,Uganda,Uganda,60000.0,90000.0,300000.0,79,Volcano Coffee stands on three business compon...,Volcano Coffee is a single source brand in Uga...,Not available,Gerald,Katabazi,gkatabazi@gmail.com,+256772992621,NaN
http://MHFM.BUSINESS.SITE_mwakyambiki_jacqueline,Mbeya Highlands Fm Radio,agriculture,Tanzania,Tanzania,189000.0,235000.0,212000.0,133,Mbeya Highlands FM Radio is a radio station th...,Mbeya Highlands FM carried out different proje...,CRDB BANK TANZANIA,Mwakyambiki,Jacqueline,jlmbiki@live.com,+255752007503,+255752007503
http://www.visit-cameroun.com_mamoudou_nene,SCS AFRIQUE CENTRALE INNOV,education,Cameroun,Cameroun,NaN,122172000.0,57573326.0,25,SCS Afrique centrale INNOV est une co-entrépri...,Les Cameroun est un Pays qui a un déficit des ...,le Promoteur SCS Afrique Centrale INNOV recher...,MAMOUDOU,NENE,nmamoudou@reseau-cioa.com,690381818,651626662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://www.mpetchad.com_youssoufrahma_amane,Association Bet Al Nadjah,education,Tchad,Tchad,300000.0,200000.0,600000.0,77,"En effet, grâce à l’appui technique et financi...",Accompagnement Technique des porteurs de proje...,NaN,Youssouf Rahma,AMANE,amane@mpetchad.com,+23566270550,NaN
http://berag-ziane.com_ziane_hafidhou,SNC BERAG ZIANE et CIE,sante,Algérie,Algérie,NaN,NaN,380000.0,154,Maîtrise d'œuvre dans le domaine du bâtiment e...,Projet : LES INTRUSIONS DANS LES HÔPITAUX ...,Aucun.-----NA-----NA,ZIANE,HAFIDHOU,ziane-hafidou@hotmail.com,213676363669,21324798024
http://www.cbdibaong.org_bigo_sedjrogwladysbertille,Etablissement AFFOKPAMA ET FILS,agriculture,Bénin,Bénin,172458.0,258565.0,262987.0,197,Créée en 2004 et agréée en 2015 par ABOMEY N° ...,Noix palmiste issu du concassage de noix de pa...,RENACA-BENIN ( Institution de financement)/CAV...,BIGO,SEDJRO GWLADYS BERTILLE,gbezinkon1@yahoo.fr,22967439837,NaN


In [22]:
export_notes_all_critere_ca2019 = notes_all_critere_ca2019.join(ALL[headers2+headers], lsuffix = '_')[headers3+headers2+headers]
export_notes_all_critere_ca2019

,FG - Solution numérique,PT - Solution numérique,AA - Solution numérique,Solution numérique - Moy,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,ca_2019,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,email_pers,mobile_pers,fixe_pers
key_main,,,,,,,,,,,,,,,,,,,,
http://MHFM.BUSINESS.SITE_mwakyambiki_jacqueline,3,3,3,3,Mbeya Highlands Fm Radio,agriculture,Tanzania,Tanzania,189000.0,235000.00,212000.00,133,Mbeya Highlands FM Radio is a radio station th...,Mbeya Highlands FM carried out different proje...,CRDB BANK TANZANIA,Mwakyambiki,Jacqueline,jlmbiki@live.com,+255752007503,+255752007503
http://www.MandulisEnergy.com_nyeko_peterbenhurodokonyero,0,1,0,"0,3333333333",Mandulis Energy,energie,Uganda,Uganda,NaN,100000.00,280000.00,101,Mandulis develops renewable energy projects in...,Mandulis Energy develops and operates off-grid...,o\tUNCDF \r\no\tTOTAL \r\no\tLaudato Si’ Chall...,Nyeko,Peter BenHur Odokonyero,Peter.Nyeko@MandulisEnergy.com,+256702347023,NaN
http://www.brastorne.com_martin_stimela,4,4,5,"4,333333333",Brastorne enterprises,agriculture,Botswana,Botswana,350000.0,570000.00,599250.00,82,Brastorne Enterprises is a 100% youth owned co...,Traditionally known for mobile wallets and pay...,NaN,Martin,Stimela,martin@brastorne.com,+26772820009,+2673114683
http://www.coldhubs.com_ikegwuonu_nnaemeka,0,0,0,0,ColdHubs Limited,agriculture,Nigeria,Nigeria,32130.0,55534.00,237000.00,63,ColdHubs Ltd. is a social enterprise that desi...,"Our 100% solar powered walk-in cold rooms, Col...",German International Cooperation (GIZ); Global...,Ikegwuonu,Nnaemeka,nnaemeka@coldhubs.com,+2348060292346,+2348060292346
http://www.havenhillsynergy.com_odunaiya_olusegun,0,1,1,"0,6666666667",HAVENHILL SYNERGY LIMITED,energie,Nigeria,Nigeria,282466.0,210570.00,404069.00,128,Havenhill Synergy is a clean-tech micro utilit...,"Specifically, we have built and operating a 2...",USADF: United States African Development Found...,Odunaiya,Olusegun,corporate@havenhillsynergy.com,+2348127736184,NaN
http://www.mellowvans.com_dupreez_neil,0,1,1,"0,6666666667",MellowVans,mobilite,South Africa,South Africa,NaN,219997.00,470000.00,80,Mellow Mobility develops and manufactures ligh...,MellowVans are electric delivery vehicles that...,SA IDC 30%\r\nKnysna Trust 19%,du Preez,Neil,neil@mellowcabs.com,+27827197904,NaN
http://www.mymdoc.com_mobisson_nneka,5,5,5,5,mDoc Healthcare,sante,Nigeria,Nigeria,NaN,NaN,201000.00,82,mDoc is a digital health social enterprise whi...,"We harness behavioural science, quality improv...",Bootstrapped and used revenue to date.,Mobisson,Nneka,nneka.mobisson@mymdoc.com,+2347063901308,NaN
http://www.paysky.io_sadek_waleed,4,5,5,"4,666666667",PaySky,finance,Egypt,Egypt,0.0,690000.00,450000.00,162,"PaySky is FinTech startup, offering a broad ra...",Proposed solution:\r\nPaySky offers a Super Pa...,Waleed Sadek\r\nHassan Sadek,Sadek,Waleed,waleed.sadek@paysky.io,+201113898888,NaN
http://www.snapnsave.co.za_fisher_tina,5,5,4,"4,666666667",SnapnSave,finance,South Africa,South Africa,214000.0,393000.00,640000.00,61,SnapnSave is South Africa’s #1 cashback coupon...,"In just over 4 years, SnapnSave has revolution...",Kalon Venture Partners\r\nVunani Investsments ...,Fisher,Tina,tina@snapnsave.co.za,+27784219892,NaN


In [23]:
DA_IO.Gspreadsheet('BRIDGE - SU potential').create()

Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com


In [24]:
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_all_critere.fillna(' '), sheet_name='Match les critères 3 CA')
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_notes_all_critere.fillna(' '), sheet_name='SU avec notes match les critères 3 CA')
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_all_critere_ca2019.fillna(' '), sheet_name='Match CA 2019')
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_notes_all_critere_ca2019.fillna(' '), sheet_name='SU avec notes match CA 2019')

Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0
Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0
Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0
Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0


In [25]:
print(len(export_all_critere))
print(len(export_notes_all_critere))
print(len(export_all_critere_ca2019))
print(len(export_notes_all_critere_ca2019))

62
6
181
14


In [26]:
export_notes_all_critere.fillna('')

,FG - Solution numérique,PT - Solution numérique,AA - Solution numérique,Solution numérique - Moy,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,ca_2019,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,email_pers,mobile_pers,fixe_pers
key_main,,,,,,,,,,,,,,,,,,,,
http://www.brastorne.com_martin_stimela,4,4,5,"4,333333333",Brastorne enterprises,agriculture,Botswana,Botswana,350000.0,570000.0,599250.0,82,Brastorne Enterprises is a 100% youth owned co...,Traditionally known for mobile wallets and pay...,,Martin,Stimela,martin@brastorne.com,+26772820009,+2673114683
http://www.havenhillsynergy.com_odunaiya_olusegun,0,1,1,"0,6666666667",HAVENHILL SYNERGY LIMITED,energie,Nigeria,Nigeria,282466.0,210570.0,404069.0,128,Havenhill Synergy is a clean-tech micro utilit...,"Specifically, we have built and operating a 2...",USADF: United States African Development Found...,Odunaiya,Olusegun,corporate@havenhillsynergy.com,+2348127736184,
http://www.snapnsave.co.za_fisher_tina,5,5,4,"4,666666667",SnapnSave,finance,South Africa,South Africa,214000.0,393000.0,640000.0,61,SnapnSave is South Africa’s #1 cashback coupon...,"In just over 4 years, SnapnSave has revolution...",Kalon Venture Partners\r\nVunani Investsments ...,Fisher,Tina,tina@snapnsave.co.za,+27784219892,
https://ozow.com/_adamseckstein_mitchanlyle,4,5,5,"4,666666667",Ozow,finance,South Africa,South Africa,334524.0,1175854.0,2175878.0,74,Ozow is the future of payment as we know it. A...,An automated EFT solution that helps customers...,"Kalon Venture Capital, Vunani, Buffet Investme...",Adams-----Eckstein,Mitchan-----Lyle,mitch@ozow.com-----lyle@ozow.com,+27 82 400 4423,+27 11 054 4744
https://rabboni.co.ug/_makumbi_henry,4,4,3,"3,666666667",Rabboni Group Ltd,agriculture,Uganda,Uganda,303630.0,508966.0,1082769.0,70,Rabboni Group Ltd is an agricultural developme...,Value addition for maize grain into human food...,KIVA (Crowd funding platform).\r\nFACTS (Finan...,Makumbi,Henry,henry@rabboni.co.ug,+256752436536,+256392962193
https://www.geodatadesign.co.za/_mentz_valerie,4,5,5,"4,666666667",GEO Data Design,technologie,South Africa,South Africa,1000000.0,1100000.0,1200000.0,245,GEO Data Design helps organisations across Afr...,Prometeo™ is a satellite-based vegetation fire...,Private Company,Mentz,Valerie,val@datadesign.co.za,+(27) 82 554 5511,+(27) 21 425 9833


In [27]:

DA_IO.Gspreadsheet('BRIDGE - SU potential').create()

Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com


In [129]:
import pandas 
v1 = '/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/Challenge_des_1_000_latest.xlsx'
#laureats = '/Users/py3/Downloads/Liste des candidats  WIIN.xlsx'
latest_challenge = pandas.read_excel(v1, sheet_name = None) 
def from_wiin(element):
    h = element.values.tolist()[0]
    data = element.values.tolist()[1:]
    return pandas.DataFrame(data, columns = h)

In [136]:
{k: from_wiin(latest_challenge[k]) for k in latest_challenge}
all_data = pandas.concat([from_wiin(latest_challenge[k]) for k in latest_challenge])

In [137]:
all_data[['Reference', 'Status']].groupby('Status').nunique()

,Reference,Status
Status,,
Candidat,73,1
Lauréat,848,1
Pré-inscrit,2056,1
À attribuer,149,1
À évaluer,127,1
Évalué,280,1


In [142]:
all_data[['Reference', 'Status', 'Courriel']].set_index('Courriel')

,Reference,Courriel
Status,,
Candidat,73,73
Lauréat,848,848
Pré-inscrit,2056,2056
À attribuer,149,149
À évaluer,127,127
Évalué,280,280


In [147]:
export_all_critere = export_all_critere.set_index('email_pers').join(all_data[['Reference', 'Status', 'Courriel']].set_index('Courriel')).fillna(' ')
export_notes_all_critere = export_notes_all_critere.set_index('email_pers').join(all_data[['Reference', 'Status', 'Courriel']].set_index('Courriel')).fillna(' ')
export_all_critere_ca2019 = export_all_critere_ca2019.set_index('email_pers').join(all_data[['Reference', 'Status', 'Courriel']].set_index('Courriel')).fillna(' ')
export_notes_all_critere_ca2019 = export_notes_all_critere_ca2019.set_index('email_pers').join(all_data[['Reference', 'Status', 'Courriel']].set_index('Courriel')).fillna(' ')

In [149]:
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_all_critere.fillna(' '), sheet_name='Match les critères 3 CA')
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_notes_all_critere.fillna(' '), sheet_name='SU avec notes match les critères 3 CA')
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_all_critere_ca2019.fillna(' '), sheet_name='Match CA 2019')
DA_IO.Gspreadsheet('BRIDGE - SU potential').df_to_sheet(export_notes_all_critere_ca2019.fillna(' '), sheet_name='SU avec notes match CA 2019')

Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0
Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0
Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0
Project ID:  web-services-d4-digital-africa
Service Account Email:  main-d4da@web-services-d4-digital-africa.iam.gserviceaccount.com
https://docs.google.com/spreadsheets/d/1Jszlt5OKIyYH6U8Gc2rBJt6kDL40YpKzZdLMI1n-hBc/edit#gid=0


In [150]:
print(len(export_all_critere))
print(len(export_notes_all_critere))
print(len(export_all_critere_ca2019))
print(len(export_notes_all_critere_ca2019))

63
6
186
14


In [151]:
export_all_critere

,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,ca_2019,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,mobile_pers,fixe_pers,Reference,Status
Kheritier@bcecoloans.com,BCECOSARL,finance,Congo-Kinshasa,Congo,257870.0,339100.0,267000.0,59,L’Entreprise « Bureau de Change Economique et ...,L’un de produits phares de BCECOSARL est l’ins...,Actuellement nous n'avons aucun investisseur. ...,KITUMAINI MUTABAZI,Héritier,+243995765138,+243852601324,d93c0dfafa,Pré-inscrit
abouathie@bimantecs.com,BIMANTECS - Bissau International Management an...,education,Guinée-Bissau,Guinée-Bissau,232976.0,386144.0,319770.0,91,BIMANTECS est une école de formations se trouv...,les filières à la quelle nous formons sont:\r\...,,ATHIE,Abou Amadou,+245955494049,+245955837853,90ac925552,Lauréat
accounts@jobartis.com,SAGINV Lda,education,Angola,Spain,400000.0,700000.0,850000.0,83,"Launched in 2013, Jobartis is the main online ...",Online Job Board: www.jobartis.com\r\nEducatio...,Business Angels\r\nThe Venture City,Verdeja,Luis,+244 933 393 226,,563093661a,Pré-inscrit
adiawara@famibmali.com,Groupe FAMIB,technologie,Mali,Mali,4342503.0,4776761.0,4726374.0,127,"Groupe Famib implanté à Bamako, Abidjan et à N...",Le Groupe FAMIB a créé UVM UPP KINGUI pour f...,,DIAWARA,Amadou,+223 75371817,+223 20290590,9b4740948e,Lauréat
ajmemergenceetfils@gmail.com,AJM EMERGENCE&FILS,agriculture,Bénin,Bénin,117166225.0,93981320.0,66848485.0,219,AJM Émergence Et Fils est un Etablissement cré...,L'huile de palmiste est de couleur jaune clair...,Banques \r\nles ONG,ADJALLALA,Jeanne,+229 97 05 06 27,,f398bc8836,Pré-inscrit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ukemeanwan2@gmail.com,Youth for Change and Social Life Development F...,agriculture,Nigeria,Nigeria,11664000.0,116640000.0,92448000.0,71,"YOUTH FOR CHANGE INITIATIVE,Youth for change &...","OCTOBER, 2018\r\nSTUDENT ORIENTATION SUMMIT\r\...",NIL,Anwan,Ukeme,+2347031177409,+2348084943878,32db472f34,Évalué
val@datadesign.co.za,GEO Data Design,technologie,South Africa,South Africa,1000000.0,1100000.0,1200000.0,245,GEO Data Design helps organisations across Afr...,Prometeo™ is a satellite-based vegetation fire...,Private Company,Mentz,Valerie,+(27) 82 554 5511,+(27) 21 425 9833,71ec95456b,Lauréat
walid-aber@hotmail.fr,Sarl ELECTRIC EXPERTS,mobilite,Algérie,Algérie,487000.0,578000.0,310000.0,116,"Notre société Sarl Electric Experts, implantée...",Notre projet c’est le développement de soluti...,,ABER AOUNI,WALID,+213541958557,+21341741268,876f7de19b,Évalué
www.mohamedabdallahhouma@gmail.com,Union maraichair de cooperative agricole d'agadez,non renseigné,Niger,Niger,585000.0,700000.0,700000.0,191,La cooperative et pour but d'ameliorè le rende...,L'oignon pomme de terr son le precipal activit...,La cooperative elle meme,Mohamed,Abdallah,+22797730729,+227,a5666f16f4,Pré-inscrit


In [152]:
export_notes_all_critere_ca2019

,FG - Solution numérique,PT - Solution numérique,AA - Solution numérique,Solution numérique - Moy,nom_struc,categorie,pays_struc1,pays_struc2,ca_2017,ca_2018,...,months,prez_struc,prez_produit_struc,investisseur_struc,nom_pers,prenom_pers,mobile_pers,fixe_pers,Reference,Status
Peter.Nyeko@MandulisEnergy.com,0,1,0,"0,3333333333",Mandulis Energy,energie,Uganda,Uganda,,100000,...,101,Mandulis develops renewable energy projects in...,Mandulis Energy develops and operates off-grid...,o\tUNCDF \r\no\tTOTAL \r\no\tLaudato Si’ Chall...,Nyeko,Peter BenHur Odokonyero,+256702347023,,b5bb737924,Lauréat
candice@solarturtle.co.za,0,1,2,1,Solar Turtle (Pty) Ltd,energie,South Africa,South Africa,2500,6619.51,...,50,The demand for accessible electricity in pay-a...,"South Africa has a large unemployment problem,...",South African National Energy Development Inst...,Tyali,Lungelwa,+27723219070,+27435550860,c4e4c3439a,Lauréat
corporate@havenhillsynergy.com,0,1,1,"0,6666666667",HAVENHILL SYNERGY LIMITED,energie,Nigeria,Nigeria,282466,210570,...,128,Havenhill Synergy is a clean-tech micro utilit...,"Specifically, we have built and operating a 2...",USADF: United States African Development Found...,Odunaiya,Olusegun,+2348127736184,,538798bf35,À évaluer
henry@rabboni.co.ug,4,4,3,"3,666666667",Rabboni Group Ltd,agriculture,Uganda,Uganda,303630,508966,...,70,Rabboni Group Ltd is an agricultural developme...,Value addition for maize grain into human food...,KIVA (Crowd funding platform).\r\nFACTS (Finan...,Makumbi,Henry,+256752436536,+256392962193,090e76fe5c,Pré-inscrit
jlmbiki@live.com,3,3,3,3,Mbeya Highlands Fm Radio,agriculture,Tanzania,Tanzania,189000,235000,...,133,Mbeya Highlands FM Radio is a radio station th...,Mbeya Highlands FM carried out different proje...,CRDB BANK TANZANIA,Mwakyambiki,Jacqueline,+255752007503,+255752007503,b29998e273,Lauréat
martin@brastorne.com,4,4,5,"4,333333333",Brastorne enterprises,agriculture,Botswana,Botswana,350000,570000,...,82,Brastorne Enterprises is a 100% youth owned co...,Traditionally known for mobile wallets and pay...,,Martin,Stimela,+26772820009,+2673114683,dd3c61dd53,Lauréat
mitch@ozow.com-----lyle@ozow.com,4,5,5,"4,666666667",Ozow,finance,South Africa,South Africa,334524,1.17585e+06,...,74,Ozow is the future of payment as we know it. A...,An automated EFT solution that helps customers...,"Kalon Venture Capital, Vunani, Buffet Investme...",Adams-----Eckstein,Mitchan-----Lyle,+27 82 400 4423,+27 11 054 4744,,
neil@mellowcabs.com,0,1,1,"0,6666666667",MellowVans,mobilite,South Africa,South Africa,,219997,...,80,Mellow Mobility develops and manufactures ligh...,MellowVans are electric delivery vehicles that...,SA IDC 30%\r\nKnysna Trust 19%,du Preez,Neil,+27827197904,,c7ad440c0e,Lauréat
nnaemeka@coldhubs.com,0,0,0,0,ColdHubs Limited,agriculture,Nigeria,Nigeria,32130,55534,...,63,ColdHubs Ltd. is a social enterprise that desi...,"Our 100% solar powered walk-in cold rooms, Col...",German International Cooperation (GIZ); Global...,Ikegwuonu,Nnaemeka,+2348060292346,+2348060292346,75cade89cf,À évaluer
nneka.mobisson@mymdoc.com,5,5,5,5,mDoc Healthcare,sante,Nigeria,Nigeria,,,...,82,mDoc is a digital health social enterprise whi...,"We harness behavioural science, quality improv...",Bootstrapped and used revenue to date.,Mobisson,Nneka,+2347063901308,,217b37a025,Lauréat
